# E4CV:fourc 4-circle example

Compare with data acquired using SPEC

In _hkl_ *E4CV* geometry (https://people.debian.org/~picca/hkl/hkl.html#org7ef08ba):

<img src="3S+1D.png" alt="E4CV geometry" width="30%"/>

* xrays incident on the $\vec{x}$ direction (1, 0, 0)

axis  | moves    | rotation about axis
---   | :---     | :---
omega | sample   | $-\vec{y}$ `[0 -1 0]`
chi   | sample   | $\vec{x}$ `[1 0 0]`
phi   | sample   | $-\vec{y}$ `[0 -1 0]`
tth   | detector | $-\vec{y}$ `[0 -1 0]`

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gi
gi.require_version('Hkl', '5.0')
from hkl.diffract import E4CV
from hkl.util import Lattice

from ophyd import (PseudoSingle, SoftPositioner)
from ophyd import Component as Cpt

In SPEC *fourc* geometry (https://certif.com/spec_help/fourc.html):

name    | mnemonic   | description
-----   | -----      | -----
2theta  | tth        | Detector arm rotation
Omega   | om         | Rotates sample circles
Chi     | chi        | Sample tilt
Phi     | phi        | Sample rotation

Same names and meanings between E4CV and SPEC but default order is different.

In [2]:
from spec2nexus.spec import SpecDataFile

specfile = SpecDataFile('hkl_data/LNO_LAO_s14.dat')
specscan = specfile.getScan(14)
print("SPEC scanCmd:", specscan.scanCmd)


SPEC scanCmd: hklscan  1.00133 1.00133  1.00133 1.00133  2.85 3.05  200 -400000


In [3]:
class Diffractometer(E4CV):
    h = Cpt(PseudoSingle, '')
    k = Cpt(PseudoSingle, '')
    l = Cpt(PseudoSingle, '')

    # use the SPEC axis names here
    omega = Cpt(SoftPositioner)
    chi = Cpt(SoftPositioner)
    phi = Cpt(SoftPositioner)
    tth = Cpt(SoftPositioner)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for p in self.real_positioners:
            p._set_position(0)  # give each a starting position

In [4]:
fourc = Diffractometer("", name="fourc")

term | value
:--- | :---
sample | LNO_LAO
crystal |  3.781726143 3.791444574 3.79890313   90.2546203 90.01815424 89.89967858
geometry | fourc
energy | 9.99906 keV
lambda | 1.239424258 Angstrom
r1 | (0 0 2) 38.09875 19.1335 90.0135 0
r2 | (1 1 3) 65.644 32.82125 115.23625 48.1315
Q | (1.001328179 1.001328179 2.999452893)
UB[0] | -1.658712442 0.09820024135 -0.000389705578
UB[1] | -0.09554990312 -1.654278629 0.00242844486
UB[2] | 0.0002629818914 0.009815746824 1.653961812

In [5]:
# add the sample to the calculation engine
fourc.calc.new_sample(
    "LNO_LAO",
    lattice=Lattice(
        a=3.781726143, b=3.791444574, c=3.79890313,
        alpha=90.2546203, beta=90.01815424, gamma=89.89967858)
    )

HklSample(name='LNO_LAO', lattice=LatticeTuple(a=3.781726143, b=3.791444574, c=3.79890313, alpha=90.2546203, beta=90.01815424, gamma=89.89967858), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.66146225e+00, -2.89938471e-03,  5.11196668e-04],
       [ 0.00000000e+00,  1.65721725e+00,  7.34922202e-03],
       [ 0.00000000e+00,  0.00000000e+00,  1.65394723e+00]]), reflections=[])

In [6]:
fourc.calc.wavelength = 1.239424258 # Angstrom

r1 = fourc.calc.sample.add_reflection(
    0, 0, 2, 
    position=fourc.calc.Position(
        tth=38.09875,
        omega=19.1335,
        chi=90.0135, 
        phi=0,
        )
    )
r2 = fourc.calc.sample.add_reflection(
    1, 1, 3, 
    position=fourc.calc.Position(
        tth=65.644,
        omega=32.82125,
        chi=115.23625, 
        phi=48.1315,
        )
    )
fourc.calc.sample.compute_UB(r1, r2)
print("SPEC UB", np.array([[-1.658712442, 0.09820024135, -0.000389705578],
    [-0.09554990312, -1.654278629, 0.00242844486],
    [0.0002629818914, 0.009815746824, 1.653961812]])
)
print(fourc.UB.get())

SPEC UB [[-1.65871244e+00  9.82002413e-02 -3.89705578e-04]
 [-9.55499031e-02 -1.65427863e+00  2.42844486e-03]
 [ 2.62981891e-04  9.81574682e-03  1.65396181e+00]]
[[-9.55499011e-02 -1.65427863e+00  2.42844485e-03]
 [ 2.62981975e-04  9.81483906e-03  1.65396181e+00]
 [-1.65871244e+00  9.82002396e-02 -3.89705577e-04]]


In [7]:
print('calc.energy is', fourc.calc.energy)
print('calc.wavelength is', fourc.calc.wavelength)
print('sample is', fourc.calc.sample)
print('position is', fourc.position)

print('sample name is', fourc.sample_name.get())
print('U matrix is', fourc.U.get(), fourc.U.describe())
print('UB matrix is', fourc.UB.get(), fourc.UB.describe())
print('reflections:',
        fourc.reflections.get(),
        fourc.reflections.describe())
print('ux is', fourc.ux.get(), fourc.ux.describe())
print('uy is', fourc.uy.get(), fourc.uy.describe())
print('uz is', fourc.uz.get(), fourc.uz.describe())
print('lattice is', fourc.lattice.get(), fourc.lattice.describe())
print(fourc.read())

calc.energy is 10.00337045202483
calc.wavelength is 1.239424258
sample is HklSample(name='LNO_LAO', lattice=LatticeTuple(a=3.781726143, b=3.791444574, c=3.79890313, alpha=90.2546203, beta=90.01815424, gamma=89.89967858), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=-90.01046218238135, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.3393109739136075, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=93.29692982869366, fit=True, inverted=False, units='Degree'), U=array([[-5.75095227e-02, -9.98327393e-01,  5.92205907e-03],
       [ 1.58283449e-04,  5.92275876e-03,  9.99982448e-01],
       [-9.98344945e-01,  5.75094506e-02, -1.82596327e-04]]), UB=array([[-9.55499011e-02, -1.65427863e+00,  2.42844485e-03],
       [ 2.62981975e-04,  9.81483906e-03,  1.65396181e+00],
       [-1.65871244e+00,  9.82002396e-02,

In [7]:
print(fourc.engine.modes)
print("(002) ?", fourc.inverse((19.1335, 90.0135, 0, 38.09875,)))
print("(113) ?", fourc.inverse((32.82125, 115.23625, 48.1315, 65.644,)))
fourc.engine.mode = "bissector"
fourc.calc["tth"].limits = (-2, 180)
print("(002) :", fourc.forward((0, 0, 2)))
print("(113) :", fourc.forward((1, 1, 3)))

['bissector', 'constant_omega', 'constant_chi', 'constant_phi', 'double_diffraction', 'psi_constant']
(002) ? DiffractometerPseudoPos(h=1.401801438592437e-16, k=0.0, l=2.0007426598535667)
(113) ? DiffractometerPseudoPos(h=1.0013281774868752, k=1.0013281774868745, l=2.99945344960595)
(002) : PosCalcE4CV(omega=19.042031585577714, chi=90.08520132331137, phi=-80.88316842824068, tth=38.08406317115543)
(113) : PosCalcE4CV(omega=32.818493106453474, chi=115.20291269085513, phi=48.13304737491596, tth=65.63698621290695)
